![idea card](images/card_lunar_new_year_rat.svg)

## Installation and Overview

At this early stage in the project, please install from GitHub. Either clone the repository or use the ```pip``` command below. 

```pip install git+https://github.com/rn123/Calendrical-Tools#egg=Calendrical-Tools```
 
Once the package is installed, the three lines below are the minimal require to **generate and print** a calendar formated as a stacked list of ISO week numbers and weeks. 

```
from calendrical_tools import candybar
cal = candybar.TextCandyBar(2020)
cal.prcandybar()```

Besides the ```Calendrical Tools``` package, this project uses the fundemntal work of Reingold & Dershowitz, Calendrical Calculations. In fact, one of the main motivations of this project is part of the close reading of Reingold & Dershowitz -- being able to reproduce the calculations and figures in their work and to develop new diagrams and illustrations to explore the topics. 

Reingold & Dershowitz have a Common Lisp implementation (Calendrica 3.0) which was ported to Python 3, [```pycalcal```](https://github.com/espinielli/pycalcal).

- Reingold, Edward M. Calendrical Calculations: The Ultimate Edition. 4 edition. Cambridge ; New York: Cambridge University Press, 2018.

In [ ]:
from pycalcal import pycalcal as pcc
from calendrical_tools import candybar
from calendrical_tools.generate_astrolabe import *

# Use the jinja package to separate the formatting of the calendars and diagrams 
# (e.g. LaTex and SVG formats) from the calendrical computations.
from jinja2 import Template

from svgpathtools import Path, Line, svg2paths
from svgpathtools import svg2paths, wsvg, disvg
import PIL

import math
import pandas as pd
from collections import namedtuple
from IPython.core.display import SVG, Image

In [ ]:
# Hack: pip commands used in development and testing.
!pip uninstall calendrical_tools -y
!pip install git+https://github.com/rn123/Calendrical-Tools#egg=Calendrical-Tools

## Text CandyBar

A plain text candybar is the default output. When the code is first run for a new year, a file containing lunar data will be generate which cound take a minute.

In [ ]:
year = 2020
cal = candybar.TextCandyBar(year=year, weeks_before=1)

Generate a calendar and display it as plain text. The current default displays the Gregorian calendar for year, with all of the weeks of the year stacked one above the other. Also, the calendar prints out the ISO week number on the left. New moons are displayed as ```NM```.

<pre>
52	23 24 25 NM 27 28 29
 1	30 31  1  2  3  4  5
 2	 6  7  8  9 10 11 12
 3	13 14 15 16 17 18 19
 4	20 21 22 23 NM 25 26
 5	27 28 29 30 31  1  2
 6	 3  4  5  6  7  8  9
 7	10 11 12 13 14 15 16
 8	17 18 19 20 21 22 NM
 9	24 25 26 27 28 29  1
</pre>

**TODO:**
- Need more consistent interface to the different formatting classes.
- Need options to generate calendrical data for larger and shorter time periods.

In [ ]:
%%capture capture --no-stderr
# Hack, but useful, to grab textual output from command.

cal.prcandybar()

In [ ]:
print(capture.stdout)

In [ ]:
# Current default generates four calendars, debugging statement below to check 
# that four calendars were generated and each consists of the same number of weeks.
for cal_type in cal.weeks:
    print(len(cal.weeks[cal_type]), cal_type)

## SVG CandyBar

In [ ]:
cal = candybar.SvgCandyBar()

In [ ]:
for cal_type in cal.weeks:
    print(len(cal.weeks[cal_type]), cal_type)

Styling the output can be done by updating the style parameters in the candybar object. The 

In [ ]:
cal_color = {
    "iso": "grey;",
    "dim": "lightblue;",
    "highlight": "green;",
    "highlight_bold": "red;",
    "background": "yellow;"
}

cal_color = {
    "iso": "#cc232a; opacity: 0.5;",
    "dim": "#cc232a;",
    "highlight": "#f5ac27;",
    "highlight_bold": "#cc232a;",
    "background": "#a3262a;"
}

cal.bar_heading = ""
cal.cal_color = cal_color
cal.prcandybar()

In [ ]:
SVG(cal.svg)

## Astrolabe Diagram

The ```Astrolabe``` class computes a diagram providing a local view of the sky. The main parameter is the ```latitude``` of the location. To facilitate travelers, early astrolabes were constructed with with exchangeble 
plates. Quoting James Morrison:

> The earliest astrolabes, which were deeply influenced by Greek tradition,
    included plates for the latitudes of the *climates.* The climates of the world
    were defined by Ptolemy to be the latitudes where the lenght of the longest
    day of the year varied by one-half hour. Ptolemy calculated the latitude
    corresponding to a 15-minute difference in the length of the longest day
    (using a value of 23 degrees 51 minutes 20 seconds for the obliquity of
    the ecliptic) for 39 latitudes, which covered the Earth from the equator
    to the North Pole. The ones called the classic *climata* were for the
    half-hour differences in the longest day covering the then populated world."""

In [ ]:
plate_parameters = {"Hawaiian Islands": 21.3069}
astrolabe = Astrolabe(plate_parameters=plate_parameters)
plate = astrolabe.plates["Hawaiian Islands"]

The current version inclues a short animation, showing the motion of the ecliptic across the local sky.

In [ ]:
animation_parameters = {"from": "0", "to": "233", "begin": "0s", "dur": "5s"}

with open("../calendrical_tools/astrolabe_template.svg") as fp:
    template_text = fp.read()

In [ ]:
# Use Inkscape extensions to svg to place different parts of astrolabe into their own layer.
inkscape_attributes = {
    identifier: 'inkscape:label="{}" inkscape:groupmode="layer"'.format(identifier)
    for identifier in identifiers
}

In [ ]:
ecliptic={
        "cx": astrolabe.xEclipticCenter,
        "cy": astrolabe.yEclipticCenter,
        "r": astrolabe.RadiusEcliptic,
        "width": 5,
    }

In [ ]:
outer_radius = ecliptic["r"] 
inner_radius = ecliptic["r"] - ecliptic["width"]

top_middle_outer =    {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] + outer_radius)}
bottom_middle_outer = {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] - outer_radius)}

top_middle_inner =    {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] + inner_radius)}
bottom_middle_inner = {"x":(ecliptic["cx"]), "y":(ecliptic["cy"] - inner_radius)}

In [ ]:
aries_first_point = astrolabe.ecliptic_division(180)
aries_first_point_angle = math.degrees(
    math.atan2(aries_first_point["y2"], aries_first_point["x2"])
)

In [ ]:
ecliptic_divisions = []
for angle in list(range(0, 361, 30)):
    ecliptic_divisions.append(astrolabe.ecliptic_division(angle))

In [ ]:
seasonal_arcs = []

seasonal_names = [    
    "雨水",
    "大寒",
    "冬至",
    "小雪",
    "霜降",
    "秋分",
    "处暑",
    "大暑",
    "夏至",
    "小满",
    "谷雨",
    "春分",
]

seasonal_names = [
        "pisces",
        "aquarius",
        "capricorn",
        "sagittarius",
        "scorpio",
        "libra",
        "virgo",
        "leo",
        "cancer",
        "gemini",
        "taurus",
        "aries",
    ]

angle = 0
for n, division in enumerate(ecliptic_divisions[0:12]):
    tag = "season" + str(angle)
    angle += 30
    next_division = ecliptic_divisions[(n + 1) % 12]
    sarc = Path(
        Arc(
            start=complex(division["x2"], division["y2"]),
            radius=complex(ecliptic["r"], ecliptic["r"]),
            rotation=0.0,
            large_arc=True,
            sweep=False,
            end=complex(next_division["x2"], next_division["y2"]),
        )
    )
    seasonal_arcs.append(
        {
            "tag": tag,
            "name": seasonal_names[n],
            "r": ecliptic["r"],
            "start_x": division["x2"],
            "start_y": division["y2"],
            "end_x": next_division["x2"],
            "end_y": next_division["y2"],
            "reversed": sarc.reversed().d(),
        }
    )

In [ ]:
stars = [
    {"name":"aldebaran",  "r": 0.7467, "theta": 68.98},
    {"name":"altair",     "r": 0.8561, "theta": 297.69542},
    {"name":"arcturus",   "r": 0.7109, "theta": 213.91500},
    {"name":"capella",    "r": 0.4040, "theta": 79.17208},
    {"name":"sirius",     "r": 1.3099, "theta": 101.28708},
    {"name":"procyon",    "r": 0.9127, "theta":114.82542},
    {"name":"deneb",      "r": 0.4114, "theta": 310.35750},
    {"name":"castor",     "r": 0.5556, "theta": 113.64958},
    {"name":"regulus",    "r": 0.8103, "theta": 152.09250},
    {"name":"vega",       "r": 0.4793, "theta": 279.23417},
    {"name":"betelgeuse", "r": 0.8784, "theta": 88.79292},
    {"name":"rigel",      "r": 1.1463, "theta": 78.63417},
    {"name":"bellatrix",  "r": 0.8949, "theta": 81.28250},
    {"name":"antares",    "r": 1.5870, "theta": 247.35167},
    {"name":"spica",      "r": 1.2096, "theta": 201.29792}
]

for star in stars:
    star["cx"] = astrolabe.RadiusEquator * star["r"] * math.cos(math.radians(star["theta"]))
    star["cy"] = astrolabe.RadiusEquator * star["r"] * math.sin(math.radians(star["theta"]))

In [ ]:
ecliptic_divisions = []
for angle in list(range(0, 361, 30)):
    ecliptic_divisions.append(astrolabe.ecliptic_division(angle))

ecliptic_divisions_fine = []
for angle in list(range(0, 361, 10)):
    ecliptic_divisions_fine.append(astrolabe.ecliptic_division(angle))

ecliptic_divisions_efine = []
for angle in list(range(0, 361, 2)):
    ecliptic_divisions_efine.append(astrolabe.ecliptic_division(angle))

In [ ]:
template = Template(template_text)
astrolabe_svg = template.render(
    place_name=plate["location"],
    latitude=plate["latitude"],
    RCapricorn=astrolabe.RadiusCapricorn,
    REquator=astrolabe.RadiusEquator,
    RCancer=astrolabe.RadiusCancer,
    horiz=plate["horizon"],
    almucantor_coords=plate["almucantars"],
    almucantar_center=plate["almucantar_center"],
    azimuth_coords=plate["azimuths"],
    prime_vertical=plate["prime_vertical"],
    ticks=astrolabe.ticks,
    ecliptic=ecliptic,
    ecliptic_divisions=ecliptic_divisions,
    ecliptic_divisions_fine=ecliptic_divisions_fine,
    ecliptic_divisions_efine=ecliptic_divisions_efine,
    aries_first_point=aries_first_point,
    aries_first_point_angle=astrolabe.obliquity,
    top_middle_outer=top_middle_outer,
    bottom_middle_outer=bottom_middle_outer,
    outer_radius=outer_radius,
    inner_radius=inner_radius,
    top_middle_inner=top_middle_inner,
    bottom_middle_inner=bottom_middle_inner,
    ecliptic_pole=astrolabe.ecliptic_pole,
    seasonal_arcs=seasonal_arcs,
    stars=stars,
    stroke_color=cal_color["highlight"],
    background_color=cal_color["highlight"],
    graph_color=cal_color["highlight"],
    inkscape=inkscape_attributes,
    animation=animation_parameters,
    )

In [ ]:
SVG(data=astrolabe_svg)

## Dividing the Ecliptic

> The procedure for dividing the ecliptic is (Figure 6-7 with the following steps numbered):
1. Locate the ecliptic pole on the meridian at $R_{eq} \tan(\epsilon / 2)$ from the center.
2. Divide the equator into equal segments of longitude: 12 divisions of 30 for the entry into each zodiac sign: more divisions depending on the resolution desired.
3. Draw a line from each equator division to the ecliptic pole. The corresponding longitude point on the ecliptic is where this line intersects the ecliptic circle.
4. A tic mark on the ecliptic is drawn toward the center of the instrument.

In [ ]:
# !pip install svgwrite
# !pip install svgpathtools

In [ ]:
# paths, attributes = svg2paths('ecliptic_division.svg')

In [ ]:
# for n, attribute in enumerate(attributes):
#     if 'id' in attribute:
#         if attribute['id'] == 'eclipticCircle':
#             print(attribute['id'])
#             eclipticCircle_path = paths[n]
#             eclipticCircle_attributes = attributes[n]
#             continue
#         if attribute['id'] == 'constructionLine':
#             print(attribute['id'])
#             constructionLine = paths[n]
#             constructionLine_attributes = attributes[n]
#             continue

In [ ]:
# intersections = []
# for (T1, seg1, t1), (T2, seg2, t2) in eclipticCircle_path.intersect(constructionLine):
#     p = eclipticCircle_path.point(T1)
#     intersections.append(p)

# intersections = [i for i in intersections if i is not None]
# intersections = list(set([(p.real, p.imag) for p in intersections]))

# match_list = []
# while len(intersections) > 0:
#     p = intersections.pop()
#     l = [p]
#     for m, q in enumerate(intersections):
#         if math.isclose(p[0], q[0], abs_tol=0.01) and math.isclose(p[1], q[1], abs_tol=0.01):
#             l.append(q)
#             intersections.pop(m)
#     match_list.append(l)

# match_list = [{"x2":m[0][0], "y2":m[0][1]} for m in match_list]

In [ ]:
css = '''
        #diagramLabel {
          fill: black;
          text-anchor: end;
          font-size: 4px;
          stroke: none;
        }
        #crossHair{
          stroke: black;
          stroke-width: 1.2;
          stroke-opacity: 0.8;          
        }
        #tropicCircles {
          fill: none;
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.3;
        }
        #eclipticCircle {
          fill: none;
          stroke: gold;
          stroke-width: 2;
          stroke-opacity: 1;
        }
        #axisFigure {
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
        #division {
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
        #divisionBold {
          stroke: red;
          stroke-width: 1;
          stroke-opacity: 1;
        }
        #eclipticPole, #eclipticCenter {
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
        #constructionLine, #constructionLine4 {
          stroke-dasharray: 2 2;
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5
        }
    '''

In [ ]:
ecliptic_division_template = """
<svg id="test" viewbox="0 0 210 210" 
     width="600" height="600"
     xmlns="http://www.w3.org/2000/svg" 
     xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" 
     xmlns:xlink="http://www.w3.org/1999/xlink"
     onload="init(evt)">
<defs>
    <style type="text/css">
        #diagramLabel {
          fill: black;
          text-anchor: end;
          font-size: 4px;
          stroke: none;
        }
        #crossHair{
          stroke: black;
          stroke-width: 1.2;
          stroke-opacity: 0.8;          
        }
        #tropicCircles {
          fill: none;
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.3;
        }
        #tropicCircles:hover {
          fill: none;
          stroke: red;
          stroke-width: 1;
          stroke-opacity: 0.3;
        }
        #eclipticCircle {
          fill: none;
          stroke: gold;
          stroke-width: 2;
          stroke-opacity: 1;
        }
        #axisFigure {
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
        #division {
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
        #divisionBold {
          stroke: red;
          stroke-width: 1;
          stroke-opacity: 1;
        }
        #eclipticPole, #eclipticCenter {
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5;
        }
        #constructionLine, #constructionLine4 {
          stroke-dasharray: 2 2;
          stroke: black;
          stroke-width: 1;
          stroke-opacity: 0.5
        }
    </style>
    <clipPath id="equatorialHole" >
        <path  d="
            M0 {{ REquator }} 
            A{{ REquator }} {{ REquator }} 0 1 0 {{ 0 }} {{ -REquator }}
            A{{ REquator }} {{ REquator }} 0 1 0 {{ 0 }} {{ REquator }}z "/>
    </clipPath>
    
    <clipPath id="eclipticExterior" >
        <path  d="
            M0 {{ ecliptic.cy + ecliptic.r + 4 }} 
            A{{ ecliptic.r + 4 }} {{ ecliptic.r + 4 }} 0 1 0 {{ 0 }} {{ ecliptic.cy - ecliptic.r - 4 }}
            A{{ ecliptic.r + 4 }} {{ ecliptic.r + 4 }} 0 1 0 {{ 0 }} {{ ecliptic.cy + ecliptic.r + 4 }}z "/>
    </clipPath> 
    
    <clipPath id="eclipticHole">
        <path fill-rule="evenodd" d="
            M{{ top_middle_outer.x }} {{ top_middle_outer.y }}
            A{{ outer_radius }} {{ outer_radius }} 0 0 1 {{ bottom_middle_outer.x }} {{ bottom_middle_outer.y }}
            A{{ outer_radius }} {{ outer_radius }} 0 0 1 {{ top_middle_outer.x }} {{ top_middle_outer.y }}z
            M{{ top_middle_inner.x }} {{ top_middle_inner.y }}
            A{{ inner_radius }} {{ inner_radius }} 0 1 0 {{ bottom_middle_inner.x }} {{ bottom_middle_inner.y }}
            A{{ inner_radius }} {{ inner_radius }} 0 1 0 {{top_middle_inner.x }} {{ top_middle_inner.y }}z"/>
    </clipPath>
    
    <clipPath id="hole" >
        <path id="test" fill-rule="evenodd" d="
            M{{ -2 }} {{ RCapricorn }} 
            A{{ RCapricorn }} {{ RCapricorn }} 0 1 0 {{ 0 }} {{ -RCapricorn }}
            A{{ RCapricorn }} {{ RCapricorn }} 0 1 0 {{ 0 }} {{ RCapricorn }}z
            M-5,{{ ecliptic_pole + 5 }} 5,{{ ecliptic_pole + 5 }} 5,{{ ecliptic_pole - 5 }} -5,{{ ecliptic_pole -5 }} -5,{{ ecliptic_pole + 5 }} 0,{{ RCapricorn }}

        "/>
    </clipPath>
</defs>

<g id="diagram" style="stroke:black; stroke-width: 1;" transform="translate(102, 102), scale(1, -1)">
    <title>Ecliptic Division</title>
    <g >
        <circle id="tropicCircles" cx="0" cy="0" r="{{ RCapricorn }}"/>
        <circle id="tropicCircles" cx="0" cy="0" r="{{ REquator }}" />
        <circle id="tropicCircles" cx="0" cy="0" r="{{ RCancer }}"/>
        <text id="diagramLabel" style="text-anchor: start;" transform="scale(1,-1)"
              x="2" y="{{ REquator + 6 }}">Equator</text>
    </g>
    
    <g id="cross">
        <line x1="0" y1="-1" x2="0" y2="1"/>
        <line x1="0" y1="-1" x2="0" y2="1"/>
    </g>
    
    <g id="axisFigure">
        <line x1="0" y1="{{ -RCapricorn }}" x2="0" y2="{{ RCapricorn }}"/>
        <line x1="{{ -RCapricorn }}" y1="0" x2="{{ RCapricorn }}" y2="0"/>
    </g>
    
    <g id="eclipticDiagram">
        <path id="eclipticCircle" d="
            M0 {{ ecliptic.cy + ecliptic.r }}
            A{{ ecliptic.r }} {{ ecliptic.r }} 0 0 1 {{ ecliptic.cx }} {{ ecliptic.cy - ecliptic.r }}
            A{{ ecliptic.r }} {{ ecliptic.r }} 0 0 1 {{ ecliptic.cx }} {{ ecliptic.cy + ecliptic.r }}z"/>

        <g id="eclipticCenter">
            <line id="crossHair" x1="-5" y1="{{ ecliptic_center }}" x2="5" y2="{{ ecliptic_center }}"/>
            <line id="crossHair" x1="0" y1="{{ ecliptic_center - 5 }}" x2="0" y2="{{ ecliptic_center + 5 }}"/>
            <text id="diagramLabel" transform="scale(1,-1)"
                  x="-2" y="{{ -ecliptic_center - 2 }}">Ecliptic Center</text>
        </g>

        <g id="eclipticPole">
            <line id="crossHair" x1="-5" y1="{{ ecliptic_pole }}" x2="5" y2="{{ ecliptic_pole }}"/>
            <line id="crossHair" x1="0" y1="{{ ecliptic_pole - 5 }}" x2="0" y2="{{ ecliptic_pole + 5 }}"/>
            <text id="diagramLabel" transform="scale(1,-1)"
                  x="-2" y="{{ -ecliptic_pole - 2 }}">Ecliptic Pole</text>
        </g>

        <g>
            <title>Divide Equator</title>
            <line id="division" x1="0" y1="{{ REquator - 4 }}" x2="0" y2="{{ REquator }}"/>
            {% for angle in angles %}
                <use xlink:href="#division" transform="rotate({{ angle }})"/>
            {%- endfor %}
        </g>

        <g>
            <line id="constructionLine" style="clip-path: url(#eclipticExterior);" 
                  x1="0" 
                  y1="{{ ecliptic_pole }}" 
                  x2="{{ constructionLineEndpoint.x2 }}" 
                  y2="{{ constructionLineEndpoint.y2 }}"/>

            <line id="constructionLine4" x1="0" y1="0" 
                  x2="{{ intersection.x2 }}" 
                  y2="{{ intersection.y2 }}" />
        </g>

        <g style="clip-path: url(#eclipticHole)">
            <title>Divide Ecliptic</title>

            {% for division in ecliptic_divisions %}
                <line id="ecliptic_division" x1="0" y1="0"
                                             x2="{{ division.x2 }}" 
                                             y2="{{ division.y2 }}"/>
            {%- endfor %}
        </g>
    </g>
</g>

<text id="output" x="10" y="190" style="font-size:3pt"></text>
<script type="application/ecmascript">
    // <![CDATA[
    var txt = document.getElementById("output");
    var r = document.getElementById("constructionLine4");
    
    function init(evt) {
        var obj;
        obj = document.getElementById("diagram");
        obj.addEventListener("click", clickButton, false);
        obj.addEventListener("mousedown", startDrag, false);
        obj.addEventListener("mousemove", doDrag, false);
    }
    
    function angle(cx, cy, ex, ey) {
      var dy = ey - cy;
      var dx = ex - cx;
      var theta = Math.atan2(dy, dx); // range (-PI, PI]
      theta *= 180 / Math.PI; // rads to degs, range (-180, 180]
      //if (theta < 0) theta = 360 + theta; // range [0, 360)
      return theta;
    }
    
    function clickButton(evt) {
        var msg = r.getAttribute("x2") + ", " +
        r.getAttribute("y2") + ", " +
        r.style.getPropertyValue("stroke") + " " +
        r.style.getPropertyValue("fill") + " " +
        angle(0, 0, r.getAttribute("x2"), r.getAttribute("y2"));
        r.setAttribute("height", "30");
        printMsg(msg); 
    }
    
    function startDrag(evt) {
        var sliderId = evt.target.parentNode.getAttribute("id");
        var svg = document.getElementById('test')
        var pt = svg.createSVGPoint();
        
        pt.x = evt.clientX; 
        pt.y = evt.clientY;
        
        var ec = document.getElementById('eclipticCenter')
        
        var svgP = pt.matrixTransform(ec.getScreenCTM().inverse());
        
        printMsg(pt.x + " " + pt.y + " | " + svgP.x + " " + svgP.y + " | " + angle(0, 0, svgP.x, svgP.y));
    }
    
    function doDrag(evt) {
        var ec = document.getElementById('cross')
        var svg = document.getElementById('test')

        var pt = svg.createSVGPoint(); 
        pt.x = evt.clientX; 
        pt.y = evt.clientY;
        
        var svgP = pt.matrixTransform(ec.getScreenCTM().inverse());
        var rotAngle = angle(0, 0, svgP.x, svgP.y)
        
        printMsg(pt.x + " " + pt.y + " | " + svgP.x + " " + svgP.y + " | " + rotAngle);
        ec.setAttribute("transform", "rotate(" + rotAngle + ")");
    }
    
    function printMsg (msg){
        txt.textContent=msg;    
    }
    // ]]>
</script>
</svg>
"""

In [ ]:
constructionLineAngle = 30
x2 = astrolabe.RadiusCapricorn * math.cos(math.radians(constructionLineAngle))
y2 = astrolabe.RadiusCapricorn * math.sin(math.radians(constructionLineAngle))
constructionLine = Line(complex(0, astrolabe.ecliptic_pole), complex(x2, y2))          

In [ ]:
p = Path(constructionLine)

In [ ]:
p.d()

In [ ]:
p.reversed().d()

In [ ]:
print(x2, y2)

In [ ]:
for (T1, seg1, t1), (T2, seg2, t2) in astrolabe.ecliptic_path.intersect(constructionLine):
    p = astrolabe.ecliptic_path.point(T1)
    
intersection = {"x2":p.real, "y2":p.imag}

In [ ]:
intersection

In [ ]:
ecliptic_divisions

In [ ]:
ecliptic_divisions = []
for angle in list(range(0, 361, 30)):
    ecliptic_divisions.append(astrolabe.ecliptic_division(angle))
    
template = Template(ecliptic_division_template)
ecliptic_division_svg = template.render(
    RCapricorn=astrolabe.RadiusCapricorn,
    REquator=astrolabe.RadiusEquator,
    RCancer=astrolabe.RadiusCancer,
    ecliptic_center=astrolabe.ecliptic_center,
    ecliptic_pole=astrolabe.ecliptic_pole,
    ecliptic=ecliptic,
    stroke_color="black;",
    angles=list(range(0, 361, 30)),
    constructionLineEndpoint={"x2":x2, "y2":y2},
    intersection=intersection,
    ecliptic_divisions=ecliptic_divisions,
    top_middle_outer=top_middle_outer,
    bottom_middle_outer=bottom_middle_outer,
    outer_radius=outer_radius,
    inner_radius=inner_radius,
    top_middle_inner=top_middle_inner,
    bottom_middle_inner=bottom_middle_inner,
)

SVG(data=ecliptic_division_svg)

In [ ]:
with open('ecliptic_division.svg', 'w') as fp:
    fp.write(ecliptic_division_svg)

In [ ]:
import svgpathtools
import cairosvg
import tinycss2
from lxml.etree import ElementTree

In [ ]:
svgpathtools.__file__

In [ ]:
from defusedxml import ElementTree

In [ ]:
svg_tree = ElementTree.fromstring(ecliptic_division_svg)

In [ ]:
svg_tree.tag

In [ ]:
import cssselect2

In [ ]:
wrapper = cssselect2.ElementWrapper.from_xml_root(svg_tree)

In [ ]:
matcher = cssselect2.Matcher()

In [ ]:
with open('ecliptic_division.svg') as fp:
    tree = cairosvg.parser.Tree(file_obj=fp)
    
for c in tree.xml_tree:
    if c.tag == '{http://www.w3.org/2000/svg}defs':
        print(c.tag)
        for d in c:
            if d.tag == '{http://www.w3.org/2000/svg}style':
                break
                print(d.tag, d.attrib)
                print(d.text)

css_parsed = tinycss2.parse_stylesheet(d.text, skip_whitespace=True)
for b in css_parsed:
    pass

b = css_parsed[1]
print(b.serialize())
type(b)

In [ ]:
rules = tinycss2.parse_stylesheet(d.text, skip_whitespace=True)
for rule in rules:
    pass

In [ ]:
for rule in rules:
    selectors = cssselect2.compile_selector_list(rule.prelude)
    selector_string = tinycss2.serialize(rule.prelude)
    content_string = tinycss2.serialize(rule.content)
    payload = (selector_string, content_string)
    for selector in selectors:
        matcher.add_selector(selector, payload)

In [ ]:
for element in wrapper.iter_subtree():
    tag = element.etree_element.tag.split('}')[-1]
    print('Found tag "{}" in HTML'.format(tag))

    matches = matcher.match(element)
    if matches:
        for match in matches:
            specificity, order, pseudo, payload = match
            selector_string, content_string = payload
            print('Matching selector "{}" ({})'.format(
                selector_string, content_string))
    else:
        print('No rule matching this tag')
    print()

In [ ]:
doc = svgpathtools.Document('ecliptic_division.svg')

In [ ]:
g = doc.get_or_add_group(["eclipticCenter"])

In [ ]:
for e in tree.xml_tree.iter():
    print(e.tag)
    print(e.attrib)

In [ ]:
import json

In [ ]:
print(b.serialize())

In [ ]:
for elem in tree.xml_tree:
    print(elem)

In [ ]:
Image(cairosvg.svg2png(ecliptic_division_svg, scale=2))

## Sun, moon, and stars

<img src="images/Moonset over Kaiaka Bay.png" alt="Moonset over Kaiaka Bay." width="25%" align="right"/>
Motion of moon and sun is computed along the ecliptic. In order to plot the position of the moon and sun on the plane of the astrolabe (actually onto the rete), first convert the ecliptic longitude and latitude to equatorial coordinates, and then use the stereographic projection.

$\sin\delta = \sin\beta\cos\epsilon + \cos\beta \sin\epsilon \sin\lambda$

Since $\beta = 0$ for the sun, the formula simplifies to:

$\sin\delta = \sin\epsilon \sin\lambda$ 

The ```CandyBar``` object contains the computed new moons of the time period. This is a list of the 

In [ ]:
from skyfield import api
from skyfield import almanac
from skyfield import almanac_east_asia as almanac_ea
from skyfield.api import Topos

from PIL.ExifTags import TAGS
from PIL.ExifTags import GPSTAGS

from datetime import datetime
from datetime import timedelta
from pytz import timezone

In [ ]:
ts = api.load.timescale()
planets = api.load('de421.bsp')
earth, moon, sun = planets['earth'], planets['moon'], planets['sun']

### Geolocate Photo

In [ ]:
image = PIL.Image.open("images/Moonset over Kaiaka Bay.png")
exif = image._getexif()

In [ ]:
labeled = {}
for (key, val) in exif.items():
    labeled[TAGS.get(key)] = val

In [ ]:
dt = datetime.strptime(labeled['DateTime'], '%Y:%m:%d %H:%M:%S')
print(dt.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
geotags = {}
for (idx, tag) in TAGS.items():
    if tag == 'GPSInfo':
        if idx not in exif:
            raise ValueError("No EXIF geotagging found")

        for (key, val) in GPSTAGS.items():
            if key in exif[idx]:
                geotags[val] = exif[idx][key]

In [ ]:
def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)


coords = get_coordinates(geotags)

In [ ]:
t0 = ts.utc(2020, 1, 1)
t1 = ts.utc(2020, 12, 31)
t, y = almanac.find_discrete(t0, t1, almanac.moon_phases(planets))

df = pd.DataFrame(y)
df['phase'] = [almanac.MOON_PHASES[yi] for yi in y]
df['utc'] = t.utc_iso()

In [ ]:
t0 = ts.utc(2020, 1, 1)
t1 = ts.utc(2020, 12, 31)
t, y = almanac.find_discrete(t0, t1, almanac.seasons(planets))

for yi, ti in zip(y, t):
    print(yi, almanac.SEASON_EVENTS[yi], ti.utc_iso(' '))

In [ ]:
kaiaka = earth + Topos(latitude_degrees=coords[0], longitude_degrees=coords[1])

### Sky

In [ ]:
hawaiian = timezone('HST')
h = hawaiian.localize(dt)
t = ts.utc(h)

In [ ]:
astrometric = kaiaka.at(t).observe(moon)
apparent = kaiaka.at(t).observe(moon).apparent()
apparent.altaz()

In [ ]:
apparent.ecliptic_latlon()

In [ ]:
alpha, delta, _ = astrometric.radec()

In [ ]:
r = astrolabe.RadiusEquator * math.tan(math.radians( (90 - delta.degrees)/2.0 ))

In [ ]:
theta = alpha._degrees

<pre>
aldebaran 16.50083 degrees
right ascension 4h 35m 55.2s
</pre>

In [ ]:
cx = r * math.cos(alpha.radians)
cy = r * math.sin(alpha.radians)

In [ ]:
print(cx, cy, r)

In [ ]:
from skyfield.api import Star, Topos, load
from skyfield.data import hipparcos

with load.open(hipparcos.URL) as f:
    df = hipparcos.load_dataframe(f)

In [ ]:
bright_stars = Star.from_dataframe(df)

astrometric = earth.at(t).observe(bright_stars)
ra, dec, distance = astrometric.radec()

print('There are {} right ascensions'.format(len(ra.hours)))
print('and {} declinations'.format(len(dec.degrees)))

## Solar Terms

![idea card](images/solarterms.png)

In [ ]:
t0 = ts.utc(2019, 12, 1)
t1 = ts.utc(2019, 12, 31)
t, tm = almanac.find_discrete(t0, t1, almanac_ea.solar_terms(e))

for tmi, ti in zip(tm, t):
    print(tmi, almanac_ea.SOLAR_TERMS_ZHS[tmi], ti.utc_iso(' '))

## 鼠年大吉

In [ ]:
with open("../docs/images/rat.svg") as fp:
    rat_svg = fp.read()

In [ ]:
rat_template = """
<svg viewbox="0 0 300 300" 
     width="300" height="300" 
     xmlns="http://www.w3.org/2000/svg" 
     xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" 
     xmlns:xlink="http://www.w3.org/1999/xlink">
<defs>
    <style type="text/css">
        #983: { fill: red;
        } 
    </style>
</defs>
<g id="rat">
    <title>Year of the Rat </title>
        {{ rat }}
</g>
</svg>
"""

In [ ]:
template = Template(rat_template)
svg = template.render(rat=rat_svg, background=cal_color["background"])

In [ ]:
SVG(svg)

## Concept Card for Project

In [ ]:
card_template = """
<svg viewbox="0 0 1280 640" 
     width="1280" height="640" 
     xmlns="http://www.w3.org/2000/svg" 
     xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" 
     xmlns:xlink="http://www.w3.org/1999/xlink">
     
<g id="card">
    
    <defs>
        <g transform="scale(1.3)">
            <clipPath id="hole">
               <path d="M75 75 L 1205 75 L 1205 565 L 75 565Z" 
                     style="stroke: {{ cal_color.highlight }} 
                     fill: {{ cal_color.background }}"/>
            </clipPath>
        </g>
    </defs>
    
    <g>
        <rect x="75px" y="75px" width="1130" height="490" 
              style="stroke: {{ cal_color.highlight }} fill: {{ cal_color.background }}"/>
    </g>

    <g transform="translate(560, 138)">
        <title>New Year Greeting</title>
        <text x="65" y="40" writing-mode="tb" 
              style="font-size:60; font-family: Courier Arial, Helvetica, sans-serif; fill:{{ cal_color.highlight }} fill-opacity:1.0;">鼠年大吉
            <tspan x="0" y="40" writing-mode="tb-rl" style="font-size: 60;">恭喜發財</tspan>
        </text>
        <line x1="-30" y1="40" x2="-30" y2="320" style="stroke: {{ cal_color.highlight }} stroke-opacity: 0.2; stroke-width:3" />
        <line x1="35"  y1="40" x2="35"  y2="320" style="stroke: {{ cal_color.highlight }} stroke-opacity: 0.2; stroke-width:3" />
        <line x1="100" y1="40" x2="100" y2="320" style="stroke: {{ cal_color.highlight }} stroke-opacity: 0.2; stroke-width:3" />
    </g>
    
    <g>
        <text x="95" y="125"
              style="font-size:30; fill: {{ cal_color.highlight }};fill-opacity:1.0;">Calendrical Tools 2020</text>
        <g transform="translate(1110, 435) scale(0.55)">
            <title>Rat</title>
            {{ rat }}
        </g>
    </g>
    
    <g transform="translate(690, 70)">
        <g transform="scale(2)">
            {{ astrolabe}}
        </g>
    </g>
    
    <g style="clip-path: url(#hole);">
        <rect x="75" y="155" width="400" height="430" style="stroke: {{ cal_color.highlight }} fill:none;"/>
        <rect x="65" y="165" width="400" height="430" style="stroke: {{ cal_color.highlight }} fill:none;"/>
        <g transform="translate(75, 155)">
            {{ candybar }}
        </g>
    </g>
    
</g></svg>
""" 

In [ ]:
template = Template(card_template)
astrolabe_svg = astrolabe_svg.replace('21.3069', "21° 18' 25''")
card_svg = template.render(candybar=cal.svg, astrolabe=astrolabe_svg, 
                           rat=rat_svg, cal_color=cal_color)
SVG(card_svg)

In [ ]:
with open('card.svg', 'w') as fp:
    fp.write(card_svg)

**TODO:**
- The current ``png`` image displays the vertical strings of Chinese character incorrectl. May be an issues with ```cairosvg```.
- Not an issue, ```writing-mode``` not supported.

In [ ]:
try:
    import cairosvg
except Exception as ex:
    print('Exception {}. Install cairosvg: '.format(ex))
    !pip install cairosvg

In [ ]:
def png_from_svg(filename="card.svg"):
    with open(filename) as fp:
        card_svg = fp.read()
    
    return cairosvg.svg2png(card_svg)

In [ ]:
png = png_from_svg()
Image(png)